# Meal Planning Notebook

This notebook helps create personalized meal plans based on your physical characteristics, activity level, and macro preferences. It uses a fuzzy logic system to adjust calorie targets and selects appropriate foods from a nutrition dataset.


In [10]:
import pandas as pd

In [11]:
import sys
import os
sys.path.append('..')

from helpers.helper import generate_first_meal_plan
from helpers.calculate_bmi import calculate_bmi
from helpers.calculate_age import calculate_age

In [12]:
df = pd.read_csv("../dataset/daily_food_nutrition_dataset.csv")

In [13]:
height = float(input("Enter your height (cm): "))
weight = float(input("Enter your weight (kg): "))
gender = input("Enter your gender (male/female): ")
exercise_rate = input("Enter your exercise rate (sedentary, light, moderate, active, very active): ")

# New Inputs
dob_input = input("Enter your date of birth (YYYY-MM-DD): ")
age = calculate_age(dob_input)

macro_preference = input("Enter your macronutrient preference (high protein, low carb, balanced, keto, etc.): ")


In [14]:
bmi = calculate_bmi(weight, height)
meal_plan = generate_first_meal_plan(df, gender, bmi, exercise_rate, age, macro_preference)

Calorie adjustment factor based on BMI, activity level, and age: 0.65


In [15]:
from tabulate import tabulate

total_row = meal_plan[['Calories (kcal)', 'Protein (g)', 'Carbohydrates (g)', 'Fat (g)', 'Fiber (g)']].sum()
total_row['Category'] = 'Total'
total_row['Food_Item'] = ''

df_with_total = pd.concat([meal_plan, pd.DataFrame([total_row])], ignore_index=True)

In [16]:
print(tabulate(df_with_total[['Meal_Type', 'Category', 'Food_Item', 'Calories (kcal)', 'Protein (g)', 'Carbohydrates (g)', 'Fat (g)', 'Fiber (g)']], 
               headers='keys', tablefmt='pretty'))
print(tabulate(df_with_total[['Food_Item']], headers='keys', tablefmt='pretty'))

+---+-----------+------------+-----------+-----------------+-------------+-------------------+--------------------+-----------+
|   | Meal_Type |  Category  | Food_Item | Calories (kcal) | Protein (g) | Carbohydrates (g) |      Fat (g)       | Fiber (g) |
+---+-----------+------------+-----------+-----------------+-------------+-------------------+--------------------+-----------+
| 0 | Breakfast |   Grains   |  Quinoa   |      223.0      |    27.2     |       34.9        |        9.2         |    3.6    |
| 1 | Breakfast |   Dairy    |  Cheese   |      183.0      |     2.2     |       23.1        |        39.0        |    7.2    |
| 2 |   Lunch   |    Meat    |   Eggs    |      163.0      |     8.9     |       24.1        |        7.3         |    7.3    |
| 3 |   Lunch   | Vegetables |  Tomato   |      188.0      |    32.8     |       28.5        |        9.8         |    3.6    |
| 4 |   Lunch   |   Grains   |   Bread   |      276.0      |     1.9     |       29.7        |        11

In [17]:
from pandas.api.types import CategoricalDtype

# Define the desired order
meal_order = ['Breakfast', 'Lunch', 'Snack', 'Dinner']
meal_type_order = CategoricalDtype(categories=meal_order, ordered=True)

# Apply the order
result_df = df_with_total[['Meal_Type', 'Category', 'Food_Item']].copy()
result_df = result_df.iloc[:-1]
result_df['Meal_Type'] = result_df['Meal_Type'].astype(meal_type_order)

# Sort and export
result_df = result_df.sort_values('Meal_Type').reset_index(drop=True)
result_df.to_csv("test_result/first_meal_plan.csv", index=False)


In [18]:
# Test the refined calorie adjustment system
import sys
sys.path.append('../')
from helpers.fuzzy_system import fuzzy_calorie_adjustment
from constant import male_targets, female_targets

print("\n🔍 TESTING REFINED CALORIE ADJUSTMENT SYSTEM\n" + "="*50)

test_cases = [
    # Format: (bmi, exercise_rate, age, description)
    (17.0, 'sedentary', 22, "Underweight youth with sedentary lifestyle"),
    (22.5, 'moderate', 30, "Normal weight young adult with moderate activity"),
    (27.0, 'active', 45, "Overweight adult with active lifestyle"),
    (31.0, 'light', 65, "Obese senior with light activity"),
    (19.0, 'very active', 25, "Lean young athlete"),
    (33.0, 'sedentary', 70, "Obese senior with sedentary lifestyle")
]

for bmi, exercise, age, desc in test_cases:
    adjustment = fuzzy_calorie_adjustment(bmi, exercise, age)
    print(f"Case: {desc}")
    print(f"  BMI: {bmi:.1f}, Exercise: {exercise}, Age: {age}")
    print(f"  Adjustment Factor: {adjustment:.2f}")
    
    # Calculate example calories for male and female
    male_cal = male_targets['calories'] * adjustment
    female_cal = female_targets['calories'] * adjustment
    print(f"  Adjusted Calories: Male: {male_cal:.0f} kcal, Female: {female_cal:.0f} kcal\n")

print("\n📊 REFERENCE TABLE FROM RESEARCH DATA:")
print("- Male (50-90kg): Sedentary 1700-3050, Moderate 2050-3600, Active 2350-4200 kcal")
print("- Female (45-85kg): Sedentary 1600-2800, Moderate 1800-3300, Active 2000-3850 kcal")
print("\nThe refined system produces calorie targets that align with research data while")
print("accounting for individual factors like BMI, specific activity level, and age.")



🔍 TESTING REFINED CALORIE ADJUSTMENT SYSTEM
Case: Underweight youth with sedentary lifestyle
  BMI: 17.0, Exercise: sedentary, Age: 22
  Adjustment Factor: 1.20
  Adjusted Calories: Male: 3390 kcal, Female: 3060 kcal

Case: Normal weight young adult with moderate activity
  BMI: 22.5, Exercise: moderate, Age: 30
  Adjustment Factor: 0.99
  Adjusted Calories: Male: 2797 kcal, Female: 2524 kcal

Case: Overweight adult with active lifestyle
  BMI: 27.0, Exercise: active, Age: 45
  Adjustment Factor: 1.00
  Adjusted Calories: Male: 2825 kcal, Female: 2550 kcal

Case: Obese senior with light activity
  BMI: 31.0, Exercise: light, Age: 65
  Adjustment Factor: 1.00
  Adjusted Calories: Male: 2825 kcal, Female: 2550 kcal

Case: Lean young athlete
  BMI: 19.0, Exercise: very active, Age: 25
  Adjustment Factor: 1.00
  Adjusted Calories: Male: 2825 kcal, Female: 2550 kcal

Case: Obese senior with sedentary lifestyle
  BMI: 33.0, Exercise: sedentary, Age: 70
  Adjustment Factor: 0.65
  Adjusted 

## 🌟 Refined Nutrition System Summary

Based on the detailed calorie and activity data from research, the meal planning system has been enhanced with the following improvements:

### 1. 📊 Enhanced Activity and Calorie Data

- **More Granular Activity Levels**: Expanded from 3 to 5 physical activity levels (PAL 1.40-2.40)
- **Gender-Specific Calorie Ranges**: Separate detailed ranges for males (50-90kg) and females (45-85kg)
- **Complete Range Representation**: Min, max, and midpoint values for all activity levels

### 2. 🧠 Advanced Fuzzy Logic System

- **Enhanced BMI Classification**: 7 ranges from severely underweight to obese class III
- **Improved Age Membership**: Research-based age groups aligned with metabolic patterns
- **Comprehensive Rule Base**: 50+ specialized rules for optimal calorie adjustment

### 3. 🔬 Research-Driven Adjustments

- **Weight-Based Scaling**: Adjustments based on position within weight range
- **Age-Related Metabolism**: Accounts for metabolic changes across lifespan
- **Safety Limits**: Ensures recommendations stay within healthy ranges

These refinements ensure that meal plans are precisely tailored to individual characteristics, resulting in more accurate and personalized nutrition recommendations that align with research-based guidelines.
